# Construcción del `VIM` - El Volatility Index del Merval 
La idea del proyecto es comenzar a definir un índice similar al `VIX` pero para el mercado de acciones argentino.
A este índice lo llamaremos `VIM`(Volatility Index Merval).
Como primera iteración utilizaremos las opciones de Grupo Financiero Galicia `GGAL`, luego iremos agregando más acciones para hacerlo más representativo. 
Con contar con las opciones de `GGAL`, `YPF`, `BMA`, `PAMP`, `TGS` y alguna más será suficiente. La idea es tomar los 5 o 6 ADRs más líquidos y con una buena cantidad de opciones.

>*"CBOE VIX - White Paper: Cbow Volatility Index"*

El `VIX` se calcula de la siguiente forma:

![](vix_formula.png)

## Primera Iteración: Cálculo de la volatilidad implícita de una opción

Se define *volatilidad* como la desviación estándar de los cambios porcentuales de los precios.

Usualmente utilizamos dos medidas de la volatilidad:
1. **Volatilidad Histórica:** Se calcula con precios históricos.
2. **Volatilidad Implícita:** Se calcula a partir de la prima de las opciones. Nos da una estimación del movimiento probable futuro de los precios. Es decir, se obtiene al despejar la volatilidad en las fórmulas con las que valuamos opciones.

Nos vamos a centrar en la **Volatilidad Implícita**
La base del cálculo es con Strikes (precio de ejercicio) que se encuentren "at the money" para vencimientos a 30 días de la expiración.


Paper: "The pricing of options and corporate liabilities", Fisher Black, Myron Scholes
Paper: "Theory of rational option pricing", Robert Merton

Comenzamos por la clasica formula de Black-Scholes-Merton. El activo subyacente $S$ sigue un movimiento Browniano geometrico:

$\mathrm{d}S=\mu S \mathrm{d}t + \sigma S \mathrm{d}z$ 

$\mu$ es la tasa instantanea esperada de retorno del activo subyacente.

$\sigma$ es la tasa instantanea de volatilidad de la tasa de retorno.

$\mathrm{d}z$ es un proceso de Wiener

### La Formula
La formula original se utiliza para valuar una opcion europea sobre una accion que no paga dividendos antes de la fecha de ejercicio. En el caso de una opcion americana, se puede utilizar para valuar un call sobre una accion que no paga dividndos.

$c$ precio de un call europeo

$p$ precio de un put  europeo 

$c={SN(d_{1})-Xe^{-rT}N(d_{2})}$

$d={Xe^{-rT}N(-d_{2})-SN(-d_{1})}$

donde:

$d_{1}={{ln(S/X)+(r+\sigma ^2 /2)T}\over{\sigma \sqrt{T}}}$

$d_{2}={{ln(S/X)+(r-\sigma ^2 /2)T}\over{\sigma \sqrt{T}}}$

$S=$ Precio de la accion

$X=$ Precio de ejercicio de la opcion (strike)

$r=$ Tasa de interes libre de riesgo (risk-free rate)

$T=$ Tiempo al vencimiento (expiration date in years)

$\sigma=$ Volatilidad en el retorno del precio del subyacente

$N(x)=$ Funcion de distribucion normal acumulada 

#### La Funcion de distribucion normal acumulada
Vamos a definir esta funcion en Python `normdist(x)`


In [9]:
import numpy as np
import scipy.stats as si

def normdist(x):
    normdist = si.norm.cdf(x, 0.0, 1.0)
    return normdist

Vamos a definir la formula y valuar un call europeo:

Precio de la accion $S=$ $60

Precio de ejercicio $X=$ $65

Risk-Free Rate $r=$ 8%

Volatilidad anual $\sigma=$ 30%

Time to maturity $T=$ 0.25 (tres meses)

In [16]:
# Calculamos d1 y d2
S = 60
X = 65
r = 0.08
σ = 0.3
T = 0.25

d1 = ( (np.log(S/X)+(r+0.5*σ**2)*T) / (σ*np.sqrt(T)) )
d2 = ( (np.log(S/X)+(r-0.5*σ**2)*T) / (σ*np.sqrt(T)) )


(d1,d2)


(-0.3252847178235758, -0.47528471782357584)

In [13]:
def BlackScholesPut(S,X,r,T,σ):
    d1 = ( (np.log(S/X)+(r+0.5*σ**2)*T) / (σ*np.sqrt(T)) )
    d2 = ( (np.log(S/X)+(r-0.5*σ**2)*T) / (σ*np.sqrt(T)) )
    BlackScholesPut = X*np.exp(-r*T)*normdist(-d2) - S*normdist(-d1)
    return (BlackScholesPut)

BlackScholesPut(60,65,0.08,0.25,0.3)

5.846282209855289

In [14]:
def BlackScholesCall(S,X,r,T,σ):
    d1 = ( (np.log(S/X)+(r+0.5*σ**2)*T) / (σ*np.sqrt(T)) )
    d2 = ( (np.log(S/X)+(r-0.5*σ**2)*T) / (σ*np.sqrt(T)) )
    BlackScholesCall = S*normdist(d1) - X*np.exp(-r*T)*normdist(d2)
    return (BlackScholesCall)

BlackScholesCall(60,65,0.08,0.25,0.3)

2.1333684449162007

In [1]:

from scipy.stats import norm
from math import log, sqrt, exp

def calculate_iv(C, S, K, T, r):
    # Estimación inicial de la volatilidad
    sigma = 0.2  
    tolerance = 0.0001
    max_iterations = 1000

    for i in range(max_iterations):
        d1 = (log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * sqrt(T))
        d2 = d1 - sigma * sqrt(T)
        
        # Precio de la opción call con la volatilidad actual
        option_price = S * norm.cdf(d1) - K * exp(-r * T) * norm.cdf(d2)
        
        # Vega (derivada del precio de la opción con respecto a la volatilidad)
        vega = S * norm.pdf(d1) * sqrt(T)
        
        # Diferencia entre el precio de la opción calculado y el real
        price_difference = option_price - C
        
        # Verificar si la diferencia está dentro del rango de tolerancia
        if abs(price_difference) < tolerance:
            return sigma
        
        # Actualizar la estimación de la volatilidad
        sigma = sigma - price_difference / vega
    
    raise Exception("No se encontró la volatilidad dentro de la tolerancia especificada")

# Parámetros dados
C = 118.05
S = 400
K = 300
T = 5 / 12
r = 0.10

# Calcular la volatilidad implícita
volatilidad_implicita = calculate_iv(C, S, K, T, r)
print("Volatilidad Implícita:", volatilidad_implicita)

Volatilidad Implícita: 0.4394726511734419
